Name: İlker Kurtuluş

Number: 2018705015

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


In [3]:
import random
import pandas as pd

In [4]:
def generate_string(N):
    my_string = list()
    my_string.append(".")
    for i in range(N-1):
        rng = random.randint(0,len(alphabet)-1)
        my_string.append(alphabet[rng])
        mys = "".join(my_string)
    return mys

In [5]:
generate_string(10)

'.qnxn.aize'

In [6]:
generate_string(12)

'.ntvxpedlusi'

In [7]:
generate_string(7)

'.tfylnu'

We need to check all triplets such that estimating _ in a_c. which is P(x2 | x1 = a, x3 = c) = P(x3 = c | x2) * P(x2 | x1 = a) / sum(x2)(P(x3 = c | x2) * P(x2 | x1 = a)  --> its for marginal of P(x1 = a, x3 = c) 

In [8]:
def get_proba_from_table(first_letter, second_letter):
    return float(T[letter2idx[first_letter]][letter2idx[second_letter]])

In [9]:
def marginalx1x3(letter1, letter3):
    marg = 0
    for letter in alphabet:
        marg = marg + get_proba_from_table(letter1, letter) * get_proba_from_table(letter, letter3)
    return marg

In [10]:
def find_argmax_x2(letter1, letter3):
    probs = list()
    for letter in alphabet:
        tmp = get_proba_from_table(letter1, letter) * get_proba_from_table(letter, letter3) / marginalx1x3(letter1,letter3)
        probs.append(tmp)
    probs = pd.Series(probs, index = alphabet)
    mx = probs.max()
    return probs[probs == mx].index[0]

In [11]:
def predict_letter(test):
    if(test[0] == "_"):
        a = "." + test[1:]
    else:
        a = test
    for i in range(len(a)):
        if(a[i] == "_"):
            if(i == len(test)-1):
                x = a[i-1]
                probs = list()
                for j in range(len(alphabet)):
                    probs.append(get_proba_from_table(x, alphabet[j]))
                probs = pd.Series(probs, index = alphabet)
                predicted_letter = probs[probs == probs.max()].index[0]
                a = a[:i] + predicted_letter + a[i+1:]
            else:
                if(a[i+1] == "_"):
                    x = a[i-1]
                    probs = list()
                    for j in range(len(alphabet)):
                        probs.append(get_proba_from_table(x, alphabet[j]))
                    probs = pd.Series(probs, index = alphabet)
                    predicted_letter = probs[probs == probs.max()].index[0]
                    a = a[:i] + predicted_letter + a[i+1:]
                else:
                    letter1 = a[i-1]
                    letter3 = a[i+1]
                    predicted_letter = find_argmax_x2(letter1,letter3)
                    a = a[:i] + predicted_letter + a[i+1:]
    return a 

In [12]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

In [13]:
for test in test_strings:
    print(test + " predicted as " + predict_letter(test))

th__br__n.f_x. predicted as the.brean.fex.
_u_st__n_.to_be._nsw_r__ predicted as .ursthend.to.be.answere.
i__at_._a_h_n_._e_r_i_g predicted as in.ath.wathend.he.r.ing
q___t.___z._____t.__.___.__. predicted as qur.t.thiz.the.at.th.the.th.
